PEM Result Visualization
========================



## Introduction



Phase Equilibrium results were produced in the notebook `notebook.org/notebook.ipynb`, but only partly visualized. More comprehensive visualization will be performed here.



## Volume Stackplots



Volume stackplots for subsequent path segments in a rock's history can be placed side-by-side to produce a combined phase equilibrium history.

-   Note: the figures may appear squashed due to screen size limits. If the figures are saved directly to non-volatile storage, their layouts will be better, and can also be editted.



In [1]:
from theriak_api import TheriakAPI,TheriakOutput,group_cols,read_theriak_table
import matplotlib.pyplot as plt
import pandas as pd
import os

# Declare the path stage names that are used to access theriak output data (by providing the filename id necessary to access the stored data for each stage).
path_stages = ["cooling","serpentinisation","exhumation"]

# Load XRF compositions df.
df = pd.read_csv(os.path.join("local_data","df.csv"),index_col=0)
# Find sample names from index.
samples = df.index

def stackplot_history(sample):
    ''' Produce side-by-side stackplots (+path characterization plots) for all path segments in the 3-part history of a specified sample.

    sample | :str: | ID for the sample in the compositions df (which should be the same as the sample ID used in filenames for storing theriak output tables).

    Returns: (:matplotlib.figure.Figure:,:matplotlib.axes.Axes:)
    '''
    # Initialize figure.
    fig = plt.figure(figsize=(13.5,8))
    # Initialize and specify geometry of grid of sub-axes with 4 rows and 3 columns.
    axs_grid = fig.add_gridspec(4,3,height_ratios=[6,1,3,1],hspace=.3,wspace=.05)
    axs = axs_grid.subplots(sharex="col",sharey="row")
    # Iterate through path stages.
    for i,path_stage in enumerate(path_stages):
        # Load data for active path stage of the specified sample.
        df = read_theriak_table(os.path.join("local_data","PEM",f"{path_stage}-{sample}-loop_table"))
        # Produce mineral volume stackplots and PTX-path characterisation plots in a column of the sub-axes grid.
        theriak_output = TheriakOutput(df)
        theriak_output.characterize_output(axs=axs[:,i])
    # Remove y labels for axes columns after the first.
    for ax in axs[:,1:].flatten():
        ax.set_ylabel("")
    # Remove the smallest-temperature x tick for all labelled temperature axes (to avoid overlap in the tick label).
    for ax in axs[-1,:]:
        ax.set_xticks(ax.get_xticks()[1:])
    # Set a centered title above the middle column
    axs[0,1].set_title(sample)
    return fig,axs

# Produce mineralogy history plots for all samples.
for sample in samples:
    _,_ = stackplot_history(sample)
    # # Save the current figure as an svg file for manual editing.
    # plt.savefig(f"figure-{sample}.svg")
plt.show()

None

## Magnetite Volume on Serpentinization Path



Plotting how magnetite volume fraction changes over the course of the serpentinisation path alongside how the volume fraction of serpentine and clinochlore change can reveal different populations of magnetite/magnetite associations.



In [1]:
from theriak_output_config import *

def local_area_stack(x,y,color,ax,baseline_y=0,**plotstyle_kwargs):
    ''' Plot a "stacked area" for a single set of xy values with a baseline y defaulting to zero.

    x | :numpy.array: | x values.
    y | :numpy.array: | y values.
    color | matplotlib color specification | Color to fill in the area of the stack.
    ax | :matplotlib.axes.Axes: | Axes to produce the plot on.
    baseline_y | Numerical or :numpy.array: | y values to stack atop.
    plotstyle_kwargs | kwargs to pass to ax.plot().

    Returns: None
    '''
    # Compute the effective upper y for a stackplot.
    y = baseline_y+y
    # Plot a line at this effective upper y.
    ax.plot(x,y,c="k",**plotstyle_kwargs)
    # Fill the stack area.
    ax.fill_between(x,baseline_y,y,color=color,zorder=-100)
    return

def magnetite_silicate_plots(serp_df):
    ''' Produce comparison plots for magnetite and silicate (serpentine and clinochlore) volumes (in a column of 3 axes) along the serpentinisation path.

    Haematite is plotted as a stack above magnetite (and is intended to represent oxidized magnetite) where present.

    serp_df | :pandas.DataFrame: | Theriak output table as a pandas dataframe for one sample's serpentinisation run.

    Returns: None
    '''
    # Isolate and group mineral phase volume columns from the theriak output table.
    serp_vol_df = group_cols(TheriakOutput(serp_df).extract_volumes())
    # Normalize to volume fractions.
    serp_vol_df = serp_vol_df.div(serp_vol_df.sum(axis=1),axis=0)
    # Declare temperature as the x variable.
    x_var = serp_df[":Temperature"]
    # Initialize figure and axes grid.
    fig,axs = plt.subplots(3,1,sharex=True,height_ratios=[2.5,1,1],figsize=(6.4,3.8),constrained_layout=True)
    # Check if haematite needs to be considered in the Fe oxide plot.
    if "Hem" in serp_vol_df:
        # If so, stack the haematite above magnetite.
        local_area_stack(x_var,serp_vol_df["Hem"],mineral_colors["Hem"],axs[0],baseline_y=serp_vol_df["Mag"])
        # Set the line style between magnetite and haematite to non-solid.
        mag_linestyle = "-."
    else:
        # Otherwise set the line style bounding magnetite (only) to solid.
        mag_linestyle = "-"

    # Produce a filled area plot for magnetite.
    local_area_stack(x_var,serp_vol_df["Mag"],mineral_colors["Mag"],axs[0],linestyle=mag_linestyle)
    # Draw a vertical line at 580, representing magnetite Curie Temperature.
    axs[0].vlines(580,0,0.05,linestyle="--",color="grey",label="Mag $T_C$")
    # Set y limit for magnetite volume fraction to between 0 and (hardcoded) 0.03.
    axs[0].set_ylim([0,0.03])
    # Set x limit to exactly the max and min temperature.
    axs[0].set_xlim([max(x_var),min(x_var)])
    # Add text to the plot.
    axs[0].set_ylabel("Rock volume fraction")
    axs[0].legend()
    axs[0].set_title(f"Magnetite stabilised during serpentinisation path: {sample}")

    # Produce volume plots for the silicates serpentine and clinochlore in the lower axes.
    for i,mineral in enumerate(["Srp","Clc"]):
        local_area_stack(x_var,serp_vol_df[mineral],mineral_colors[mineral],axs[i+1])
        axs[i+1].set_ylim([0,max(serp_vol_df[mineral])*1.07])

    # Label the x axes for only the lowermost axes.
    axs[2].set_xlabel("(Decreasing) Temperature /$^{\\circ}\\text{C}$")
    return

# Iterate through samples.
for sample in samples:
    # Load theriak output table for serpentinisation path for the active sample.
    serp_df = read_theriak_table(os.path.join("local_data","PEM",f"serpentinisation-{sample}-loop_table"))
    # Produce magnetite-silicate volume comparison plots.
    magnetite_silicate_plots(serp_df)
    # plt.savefig(f"figure-{sample}-mag-stack.svg")
plt.show()

None